## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
import pickle
from datetime import datetime

import ray

import warnings
warnings.filterwarnings('ignore')


In [2]:
#from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG, APEX_DEFAULT_CONFIG
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

Import the Environment

In [3]:
sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

['id0000569']
([42449991, 42429690, 42445236, 42436486, 42438800, 42439561, 42430063, 42450009, 42446472], ['2016-01-03 19:51:38', '2016-01-03 19:51:53', '2016-01-03 19:52:08', '2016-01-03 19:52:24', '2016-01-03 19:52:40', '2016-01-03 19:52:55', '2016-01-03 19:53:10', '2016-01-03 19:53:25', '2016-01-03 19:53:40'])


In [4]:
env_config = {'pickup_hub_index':2,
        'delivery_hub_index':3,
        'pickup_timestamp': datetime(2016, 6, 14, 22, 0, 0),
        'delivery_timestamp': datetime(2016, 6, 14, 1, 0, 0) 
}

with open('env_config.pkl', 'wb') as f:
    pickle.dump(env_config, f)

In [5]:
env=GraphEnv()


Run random steps without Ray Trainer

In [6]:
def run_one_episode (env):
    env.reset()
    print("reset done")
    sum_reward = 0
    for i in range(30):
        action = env.action_space.sample()
        #print(action)
        state, reward, done, info = env.step(action)
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward)
            break

        print("sum_reward: ",sum_reward)
        # print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(1): 
    run_one_episode (env)

Reset with config
final hub: 3
current position: 2
reset done
action ==  39  New Position 39
2 -> 39 50.0
Step() Execution time: 0.2602269649505615 seconds
sum_reward:  -50.0
action ==  19  New Position 19
39 -> 19 50.0
Step() Execution time: 0.3269767761230469 seconds
sum_reward:  -100.0
action ==  51  New Position 51
19 -> 51 50.0
Step() Execution time: 0.3386270999908447 seconds
sum_reward:  -150.0
action ==  6  New Position 6
51 -> 6 50.0
Step() Execution time: 0.26720094680786133 seconds
sum_reward:  -200.0
action ==  5  New Position 5
6 -> 5 50.0
Step() Execution time: 0.5763471126556396 seconds
sum_reward:  -250.0
action ==  1  New Position 1
5 -> 1 50.0
Step() Execution time: 0.3538229465484619 seconds
sum_reward:  -300.0
action == wait 
1 -> 1 2.0
Step() Execution time: 0.597296953201294 seconds
sum_reward:  -302.0
action ==  32  New Position 32
1 -> 32 50.0
Step() Execution time: 0.38727498054504395 seconds
sum_reward:  -352.0
action ==  50  New Position 50
32 -> 50 50.0
Step

Initialize Ray

In [7]:
ray.init()

2022-05-12 02:11:23,141	INFO services.py:1412 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': '/tmp/ray/session_2022-05-12_02-11-18_781193_39419/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-12_02-11-18_781193_39419/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2022-05-12_02-11-18_781193_39419',
 'metrics_export_port': 59977,
 'gcs_address': '127.0.0.1:59256',
 'address': '127.0.0.1:59256',
 'node_id': 'ed025f11873de89f091cbe52d659fafd6641a5f0a0fa24a1b3e496e7'}

Set DQN trainer configuration

In [8]:
dqn_config = DEFAULT_CONFIG.copy()
dqn_config['num_workers'] = 1
dqn_config["train_batch_size"] = 400
dqn_config["gamma"] = 0.95
dqn_config["n_step"] = 10
dqn_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#dqn_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

Set DQN - Rainbow configuration

In [9]:
rainbow_config = DEFAULT_CONFIG.copy()

rainbow_config = DEFAULT_CONFIG.copy()
rainbow_config['num_workers'] = 1
rainbow_config["train_batch_size"] = 400
rainbow_config["gamma"] = 0.95
rainbow_config["n_step"] = 10
rainbow_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#rainbow_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

#rainbow parameters

# N-step Q learning
rainbow_config["n_step"]= 5 #[between 1 and 10]
# Whether to use noisy network
rainbow_config["noisy"] = True
# Number of atoms for representing the distribution of return. When
# this is greater than 1, distributional Q-learning is used.
# the discrete supports are bounded by v_min and v_max
rainbow_config["num_atoms"] =2 #[more than 1]
rainbow_config["v_min"] =-100
rainbow_config["v_max"]=100 # (set v_min and v_max according to your expected range of returns).

Set DQN - Ape-X configuration 

In [10]:
#apex_config=APEX_DEFAULT_CONFIG.copy()

Initialize the DQN trainer, Rainbow DQN and DQN Ape-X

In [9]:
ppo_trainer=PPOTrainer(DEFAULT_CONFIG,GraphEnv)

2022-05-12 02:11:39,268	INFO trainer.py:2140 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also then want to set eager_tracing=True in order to reach similar execution speed as with static-graph mode.
2022-05-12 02:11:39,274	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-05-12 02:11:39,275	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


(RolloutWorker pid=39477) ['id0000569']
(RolloutWorker pid=39477) ([42449991, 42429690, 42445236, 42436486, 42438800, 42439561, 42430063, 42450009, 42446472], ['2016-01-03 19:51:38', '2016-01-03 19:51:53', '2016-01-03 19:52:08', '2016-01-03 19:52:24', '2016-01-03 19:52:40', '2016-01-03 19:52:55', '2016-01-03 19:53:10', '2016-01-03 19:53:25', '2016-01-03 19:53:40'])
(RolloutWorker pid=39478) ['id0000569']
(RolloutWorker pid=39478) ([42449991, 42429690, 42445236, 42436486, 42438800, 42439561, 42430063, 42450009, 42446472], ['2016-01-03 19:51:38', '2016-01-03 19:51:53', '2016-01-03 19:52:08', '2016-01-03 19:52:24', '2016-01-03 19:52:40', '2016-01-03 19:52:55', '2016-01-03 19:53:10', '2016-01-03 19:53:25', '2016-01-03 19:53:40'])


(RolloutWorker pid=39477) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=39477) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=39478) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=39478) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=39477) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
(RolloutWorker pid=39477) UserWarning: The `get_nearest

(RolloutWorker pid=39477) Reset with config
(RolloutWorker pid=39477) final hub: 3
(RolloutWorker pid=39477) current position: 2
(RolloutWorker pid=39478) Reset with config
(RolloutWorker pid=39478) final hub: 3
(RolloutWorker pid=39478) current position: 2


2022-05-12 02:12:12,126	INFO trainable.py:127 -- Trainable.setup took 32.862 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-12 02:12:12,137	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [ ]:
trainer_dqn = DQNTrainer(dqn_config,GraphEnv)

In [ ]:
trainer_rainbow=DQNTrainer(rainbow_config,GraphEnv )


In [ ]:
#trainer_apex=DQNTrainer(apex_config,GraphEnv )

Define the path where the results of the trainer should be saved

In [ ]:
checkpoint_root = "tmp/dqn/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [15]:
results = []
episode_data = []
episode_json = []
n_iter=10

for n in range(n_iter):
    result = ppo_trainer.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = ppo_trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

StopIteration: 

In [ ]:
results

Testing of the trainer

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer_dqn.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()